In [ ]:
import requests 
import pandas as pd
import io
import matplotlib as plt
import numpy as np
from scipy import stats
import matplotlib.pyplot as plta

In [40]:
# hide and read API key from text file
def get_env_values(setup_filename):
    SETUP_FILE = open(setup_filename,'r')
    SETUP_VAR = SETUP_FILE.readlines()
    API_KEY = SETUP_VAR[0]
    TICKERS = SETUP_VAR[1:]

    return (API_KEY.strip(),TICKERS)

In [38]:
# return price information (historic) for 'symb'
def alpha_vantage_equity(symb,API_KEY):
    link = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&outputsize=full&apikey={}&datatype=csv'.format(symb,API_KEY)
    rq = requests.get(link)
    urlData = rq.content
    rawData = pd.read_csv(io.StringIO(urlData.decode('utf-8')))
    
    # returns encoded utf-8 CSV format but in a Pandas DataFrame
    return rawData

In [ ]:
#returns a array object with technical - statistical indicators 

def build_table(symb,API_KEY):
    
    data = alpha_vantage_equity(symb,API_KEY)
    # in 1 year there are 252 trading days 
    year_data = data[0:252]
    frame = year_data.sort_values(by='timestamp')
    frame.set_index('timestamp',inplace=True)
    frame['Returns'] = frame['adjusted_close'].pct_change()
    r_timeseries = frame.Returns.dropna()

    # mean
    mean = np.mean(r_timeseries)
    # annualized return
    m_annual = ((1+mean)**252)-1


    # risk measures
    vol = (np.std(r_timeseries))**2
    v_annual = np.std(r_timeseries)*np.sqrt(252)

    risk_free_rate = 1.60
    # sharpe ratio      ASSET A has AR - 207% and a risk of 50% ---- RF ER=0 and risk of 0%
    sr = (m_annual-risk_free_rate)/(v_annual/np.sqrt(252))

    out = [m_annual,v_annual,sr]
    

    return out

In [ ]:
# disambigouation for tickers
def get_ticker(keyword,API_KEY):

    link = 'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords={}&apikey={}&datatype=csv'.format(keyword,API_KEY)
    rq = requests.get(link)
    urlData = rq.content
    rawData = pd.read_csv(io.StringIO(urlData.decode('utf-8')))

    return rawData['symbol'][0]

In [42]:
def main(filename):

    ENV = get_env_values(filename)
    API_KEY, names = ENV[0], ENV[1]

    data_frame = pd.DataFrame(columns=['Mean Annual','Volatility Annual','Sharpe Ratio','Symbol'])

   
    i = 0
    # 1 per eqquity
    for equity in names:
        tmp_ticker = get_ticker(equity.strip(),API_KEY)
        time.sleep(13)
        data_point = build_table(tmp_ticker,API_KEY)
        data_point.append(tmp_ticker)
        data_frame.loc[i] = data_point
        i = i + 1

    
    return data_frame

In [43]:
main('setup.txt')

KeyError: 'symbol'